## Generate lesion(PIRADs) GT

In [1]:
import os
import copy
import SimpleITK as sitk
import numpy as np
import pandas as pd
from tqdm import tqdm
from skimage.measure import label, regionprops

In [13]:
#mask_root = rf'D:\dataset\B78_GT_output'
mask_root = rf'D:\dataset\Suzhou_GT_output'

In [14]:
def get_lesion_info(mask):
    lesion_list = []
    image = sitk.ReadImage(mask)
    mask_array = sitk.GetArrayFromImage(image)
    #print(mask_array.shape)
    labels = label(mask_array, connectivity=3)
    regions = regionprops(labels)
    for region in regions:
        centroid = np.round(region.centroid)
        phys_centroid = image.TransformContinuousIndexToPhysicalPoint(centroid[::-1])
        #print(centroid, phys_centroid)
        score = mask_array[region.coords[0][0], region.coords[0][1], region.coords[0][2]]
        lesion_list.append([phys_centroid, score])
    return lesion_list

def compute_distance(loc_1, loc_2):
    distance = ((loc_1[0] - loc_2[0])**2 + (loc_1[1] - loc_2[1])**2 + (loc_1[2] - loc_2[2])**2)**0.5
    return distance


def merge_lesion(lesion_info, threshold=2):
    lesions_list = []
    for key in lesion_info:
        lesions_list.extend(lesion_info[key])
    
    lesion_count = len(lesions_list)
    dist_matrix = np.zeros((len(lesions_list), len(lesions_list)))
    for i in range(lesion_count):
        for j in range(lesion_count):
            if i == j:
                dist_matrix[i,j] = 1000
            else:
                dist_matrix[i,j] = compute_distance(lesions_list[i][0], lesions_list[j][0])
    match_lesions = np.where(dist_matrix < threshold)
    gt_lesion = []
    if len(match_lesions[0]):
        
        for i in range(lesion_count):
            match_count = 1
            match_pair = []
            for j in range(len(match_lesions)):
                if match_lesions[0][j] == i:
                    match_count += 1
                    match_pair.append(match_lesions[1][j])
            if match_count >= threshold:
                lesion_loc_stack = np.array(lesions_list[i][0])
                lesion_score_list = [lesions_list[i][1]]
                for index in match_pair:
                    lesion_loc_stack = np.concatenate([lesion_loc_stack, np.array(lesions_list[index][0])])
                    lesion_score_list.append(lesions_list[index][1])
                #print(lesion_loc_stack)
                lesion_loc_stack = lesion_loc_stack.reshape(match_count, 3)
                merged_lesion_loc = list(np.round(np.mean(lesion_loc_stack, axis=0), 4))
                #print(merged_lesion_loc)
                merged_lesion_score = np.floor(np.median(np.array(lesion_score_list)))
                merged_lesion_info = [merged_lesion_loc, merged_lesion_score, match_count]
                
                if merged_lesion_info not in gt_lesion:
                    gt_lesion.append(merged_lesion_info)
    return gt_lesion
                
                
            
            
            
            

In [15]:
#annotators = ['WJ2', 'WTL2', 'XPY2']
annotators = ['nantong', 'shanghai', 'soochow']
#case_list = ['P' + (3-len(str(x)))*'0' + str(x) for x in range(1,181)]
case_list = ['P1' + (3-len(str(x)))*'0' + str(x) for x in range(1,301)]
mv_thres = 2
dist_thres = 10
case_info = dict()
cols = ['PID', 'Lesion_Idx', 'Phys_Z', 'Phys_Y', 'Phys_X', 'Score', 'Contri']
for col in cols:
    case_info.setdefault(col, [])
for case in tqdm(case_list):
    lesions_info = dict()
    for annotator in annotators:
        mask_file = os.path.join(mask_root, annotator, case + '.mhd')
        lesions_info[annotator] = get_lesion_info(mask_file)
    gt_lesions = merge_lesion(lesions_info)
    if gt_lesions:
        lesion_index = 0
        for gt_lesion in gt_lesions:
            case_info['PID'].append(case)
            lesion_index += 1
            case_info['Lesion_Idx'].append(lesion_index)
            case_info['Phys_Z'].append(gt_lesion[0][0])
            case_info['Phys_Y'].append(gt_lesion[0][1])
            case_info['Phys_X'].append(gt_lesion[0][2])
            case_info['Score'].append(gt_lesion[1])
            case_info['Contri'].append(gt_lesion[2])
    else:
        case_info['PID'].append(case)
        for col in ['Lesion_Idx', 'Phys_Z', 'Phys_Y', 'Phys_X', 'Score', 'Contri']:
            case_info[col].append('')
case_df = pd.DataFrame(case_info)



100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:45<00:00,  6.65it/s]


In [17]:
case_df.to_excel('suzhou_perf_pirads_gt.xlsx', index=False)

In [16]:
case_df

,PID,Lesion_Idx,Phys_Z,Phys_Y,Phys_X,Score,Contri
0,P1001,1,2.8435,-4.8512,50.573,5.0,2
1,P1002,,,,,,
2,P1003,1,12.5847,1.9775,52.5241,5.0,2
3,P1004,,,,,,
4,P1005,1,-0.4981,-7.4398,16.2315,3.0,2
...,...,...,...,...,...,...,...
309,P1296,,,,,,
310,P1297,,,,,,
311,P1298,,,,,,
312,P1299,1,-15.5935,51.8395,6.1941,4.0,2
